In [3]:
import os
import seaborn as sns
import math
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime, timedelta

In [4]:

VARIABLES = ["stl1", "tp", "swvl1", "u10", "v10", "ssr", "value_ndvi", "value_fapar"]

OPERATIONS = ['min', 'max', 'mean', 'std']

METEO_COLUMNS = [f"{variable}_{operation}" for variable in VARIABLES for operation in OPERATIONS]

N_WEEKS = 4

group_days_in_weeks =  lambda x: 1 if x == 0 else math.ceil(x - 1) // 7 + 1

main_data_folder = f"../data"
meteo_data_folder = f"{main_data_folder}/clean/final"


def read_raw_meteo_data(codparcela: str) -> pd.DataFrame:
    file_path = f"{meteo_data_folder}/{codparcela}.csv"

    if not os.path.exists(file_path):
        print(f"Error: The file {codparcela}.csv does not exist in {meteo_data_folder}.")
        return None

    df_meteo_parcela = pd.read_csv(file_path, index_col=0, sep="\t", parse_dates=["date"])
    return df_meteo_parcela



def group_meteo_data_row(fecha_muestra, df_meteo_parcela):

        fecha_end_slice = fecha_muestra - timedelta(7*N_WEEKS)

        df_meteo_sample = df_meteo_parcela[(df_meteo_parcela["date"] >= fecha_end_slice) & (df_meteo_parcela["date"] <= fecha_muestra)].copy()

        df_meteo_sample["diff_days_sample"] = (fecha_muestra - df_meteo_sample["date"]).dt.days   

        df_meteo_sample["weeks"] = df_meteo_sample["diff_days_sample"].apply(group_days_in_weeks)#, args=(n_weeks,))

        df_meteo_raw_grouped = df_meteo_sample.groupby("weeks")[VARIABLES].agg(OPERATIONS)

        df_meteo_raw_grouped.columns = METEO_COLUMNS

        return df_meteo_raw_grouped


def get_meteo_variables_row(id_codparcela, fecha_muestra, df_meteo_raw_grouped):

    n_rows, n_column = df_meteo_raw_grouped.shape

    flat_values = df_meteo_raw_grouped.values.reshape(n_rows * n_column)

    resultados_columns = [f"{column}_{i}" for i in range(1, n_rows+1) for column in df_meteo_raw_grouped.columns]

    df_meteo_proccessed = pd.DataFrame([flat_values], columns=resultados_columns)

    df_meteo_proccessed["codparcela"] = id_codparcela
    df_meteo_proccessed["fecha"] = fecha_muestra
        
    df_meteo_proccessed.insert(0, "codparcela", df_meteo_proccessed.pop("codparcela"))
    df_meteo_proccessed.insert(1, "fecha", df_meteo_proccessed.pop("fecha"))

    return df_meteo_proccessed



def get_meteo_variables_parcela(df_subsamples, df_meteo_parcela):

    df_concatenado = pd.DataFrame()

    for _, row in df_subsamples.iterrows():

        fecha_muestra = row["fecha"]

        id_codparcela = row["codparcela"]

        df_meteo_raw_grouped = group_meteo_data_row(fecha_muestra, df_meteo_parcela)

        df_meteo_proccessed = get_meteo_variables_row(id_codparcela, fecha_muestra, df_meteo_raw_grouped)
        

        if df_concatenado.empty:
            df_concatenado = df_meteo_proccessed.copy()
        else:
            df_concatenado = pd.concat([df_concatenado, df_meteo_proccessed], axis=0, ignore_index=True, keys=['df_concatenado', 'df_meteo_proccessed'])

    return df_concatenado

In [5]:


sample_filename = "subset_muestreos_parcelas.parquet"

df_samples = pd.read_parquet(f"{main_data_folder}/{sample_filename}")

df_samples["fecha"] = pd.to_datetime(df_samples["fecha"])

df_samples["codparcela"] = df_samples["codparcela"].str.replace("/", "")

df_samples.info()

<class 'pandas.core.frame.DataFrame'>
Index: 578377 entries, 0 to 581792
Data columns (total 63 columns):
 #   Column                                              Non-Null Count   Dtype         
---  ------                                              --------------   -----         
 0   generated_muestreos                                 578377 non-null  datetime64[us]
 1   codparcela                                          578377 non-null  object        
 2   provincia                                           578377 non-null  string        
 3   municipio                                           578377 non-null  string        
 4   fecha                                               578377 non-null  datetime64[us]
 5   campaña                                             578377 non-null  uint16        
 6   poligono                                            578377 non-null  UInt16        
 7   parcela                                             578347 non-null  UInt16        
 8  

In [7]:
codparcelas = df_samples["codparcela"].unique()

df_meteo_variables_parcela_total = pd.DataFrame()

for i, codparcela in enumerate(codparcelas):

    df_subsamples_parcela = df_samples[df_samples["codparcela"]==codparcela]

    df_meteo_parcela = read_raw_meteo_data(codparcela)

    df_meteo_variables_parcela = get_meteo_variables_parcela(df_subsamples_parcela, df_meteo_parcela)

    if df_meteo_variables_parcela.empty:
        df_meteo_variables_parcela_total = df_meteo_variables_parcela.copy()
    else:
        df_meteo_variables_parcela_total = pd.concat([df_meteo_variables_parcela_total, df_meteo_variables_parcela], axis=0, ignore_index=True, keys=['df_meteo_variables_parcela_total', 'df_meteo_variables_parcela'])

    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178


In [131]:



files = os.listdir(meteo_data_folder)




df_concatenado = pd.DataFrame()

for n_files, file in enumerate(files):

    filename = file.split(".csv")[0]
    df_meteo_parcela = pd.read_csv(f"{meteo_data_folder}/{file}", index_col=0, sep="\t", parse_dates=["date"])
    df_subsamples = df_samples[df_samples["codparcela"]==filename]

    i = 0
    flag = 0

    print("Parcela terminada")
    
    for _, row in df_subsamples.iterrows():

        fecha_muestra = row["fecha"]

        id_codparcela = row["codparcela"]

        df_meteo_raw_grouped = group_meteo_data(fecha_muestra, n_weeks, meteo_columns, df_meteo_parcela)

        df_meteo_proccessed = get_meteo_variables(id_codparcela, fecha_muestra, df_meteo_raw_grouped)

        # Concatenar los DataFrames verticalmente por columnas con nombres de columnas
        


        if df_concatenado.empty:
            df_concatenado = df_meteo_proccessed.copy()
        else:
            df_concatenado = pd.concat([df_concatenado, df_meteo_proccessed], axis=0, ignore_index=True, keys=['df_concatenado', 'df_meteo_proccessed'])

        # # Renombrar las columnas según el patrón "variable_stat"
        # resultados.columns = [f'duracion_dias_{stat}' for stat in resultados.columns]

        
    




Parcela terminada
Parcela terminada
Parcela terminada
Parcela terminada
Parcela terminada
Parcela terminada
Parcela terminada
Parcela terminada
Parcela terminada
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/carlossi/.cache/pypoetry/virtualenvs/zrive-ds-e6lejYny-py3.11/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3548, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_4583/2324745828.py", line 28, in <module>
    df_meteo_raw_grouped = group_meteo_data(fecha_muestra, n_weeks, meteo_columns, df_meteo_parcela)
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_4583/3101021656.py", line 17, in group_meteo_data
    df_meteo_raw_grouped = df_meteo_sample.groupby("weeks")[VARIABLES].agg(OPERATIONS)
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/carlossi/.cache/pypoetry/virtualenvs/zrive-ds-e6lejYny-py3.11/lib/python3.11/site-packages/pandas/core/groupby/generic.py", line 1445, in aggregate
    result = op.agg()
             ^^^^^^^^
  File "/home/carlossi/.cach

In [132]:
df_concatenado

,codparcela,fecha,stl1_min_1,stl1_max_1,stl1_mean_1,stl1_std_1,tp_min_1,tp_max_1,tp_mean_1,tp_std_1,...,ssr_mean_4,ssr_std_4,value_ndvi_min_4,value_ndvi_max_4,value_ndvi_mean_4,value_ndvi_std_4,value_fapar_min_4,value_fapar_max_4,value_fapar_mean_4,value_fapar_std_4
0,024-00189-08-00,2019-03-18,281.966725,281.966725,281.966725,NaN,1.862645e-09,1.862645e-09,1.862645e-09,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,024-00189-08-00,2019-03-25,278.838645,291.067814,284.347225,3.587785,1.862645e-09,7.339917e-03,6.856139e-04,0.001668,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,024-00189-08-00,2019-04-01,272.157329,291.912220,284.634421,4.047920,1.862645e-09,1.321317e-02,1.512169e-03,0.003473,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,024-00189-08-00,2019-04-08,278.034025,291.813933,284.122565,3.277489,1.862645e-09,1.321317e-02,2.598026e-03,0.003795,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,024-00189-08-00,2019-04-15,271.765743,291.922725,283.394482,3.915372,1.862645e-09,8.159934e-03,7.895863e-04,0.001999,...,2.820382e+06,3.253705e+06,0.328,0.328,0.328,NaN,0.312,0.312,0.312,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1357,012-00001-00-00,2009-08-11,271.284571,291.957395,281.089826,6.544224,1.862645e-09,1.019915e-03,6.243451e-05,0.000222,...,5.347328e+06,4.316016e+06,NaN,NaN,NaN,NaN,0.144,0.144,0.144,NaN
1358,012-00001-00-00,2009-08-17,271.270833,291.816518,279.945535,6.015677,1.862645e-09,3.858239e-03,3.216887e-04,0.000866,...,5.118401e+06,4.024069e+06,0.236,0.236,0.236,NaN,0.144,0.144,0.144,NaN
1359,012-00001-00-00,2009-08-24,271.469962,291.460134,279.401155,6.330392,1.862645e-09,5.910754e-05,2.968153e-06,0.000012,...,5.205792e+06,4.098523e+06,0.268,0.268,0.268,NaN,0.128,0.128,0.128,NaN
1360,012-00001-00-00,2009-08-31,271.330622,291.990513,280.674619,6.436609,1.862645e-09,7.837306e-05,3.823082e-06,0.000016,...,4.859498e+06,3.844316e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [128]:
df_meteo_raw_grouped

,stl1_min,stl1_max,stl1_mean,stl1_std,tp_min,tp_max,tp_mean,tp_std,swvl1_min,swvl1_max,...,ssr_mean,ssr_std,value_ndvi_min,value_ndvi_max,value_ndvi_mean,value_ndvi_std,value_fapar_min,value_fapar_max,value_fapar_mean,value_fapar_std
weeks,,,,,,,,,,,,,,,,,,,,,
1,271.379866,291.989829,284.492825,4.734898,1.862645e-09,0.012916,0.001524,0.003326,0.241688,0.423226,...,3.464083e+06,3.564969e+06,NaN,NaN,NaN,NaN,0.520,0.520,0.520,NaN
2,275.527758,291.505870,283.789412,4.234237,1.862645e-09,0.002386,0.000019,0.000198,0.288580,0.358341,...,3.052056e+06,3.729696e+06,0.684,0.684,0.684,NaN,0.496,0.496,0.496,NaN


In [129]:
df_meteo_proccessed

,codparcela,fecha,stl1_min_1,stl1_max_1,stl1_mean_1,stl1_std_1,tp_min_1,tp_max_1,tp_mean_1,tp_std_1,...,ssr_mean_2,ssr_std_2,value_ndvi_min_2,value_ndvi_max_2,value_ndvi_mean_2,value_ndvi_std_2,value_fapar_min_2,value_fapar_max_2,value_fapar_mean_2,value_fapar_std_2
0,003-00028-00-00,2006-03-20,271.379866,291.989829,284.492825,4.734898,1.862645e-09,0.012916,0.001524,0.003326,...,3.052056e+06,3.729696e+06,0.684,0.684,0.684,NaN,0.496,0.496,0.496,NaN


In [112]:

    

df_2 = get_meteo_variables(id_codparcela, fecha_muestra, resultados)

In [113]:
df_2


,codparcela,fecha,stl1_min_1,stl1_max_1,stl1_mean_1,stl1_std_1,tp_min_1,tp_max_1,tp_mean_1,tp_std_1,...,ssr_mean_2,ssr_std_2,value_ndvi_min_2,value_ndvi_max_2,value_ndvi_mean_2,value_ndvi_std_2,value_fapar_min_2,value_fapar_max_2,value_fapar_mean_2,value_fapar_std_2
0,024-00189-08-00,2019-04-01,272.157329,291.91222,284.634421,4.04792,1.862645e-09,0.013213,0.001512,0.003473,...,2.820382e+06,3.253705e+06,0.328,0.328,0.328,NaN,0.312,0.312,0.312,NaN


In [103]:
df.to_csv("test_flatten.txt", sep="\t")

In [86]:
df_datos_muestra

,date,value_ndvi,value_fapar,stl1,tp,swvl1,u10,v10,ssr,latitude,longitude,diff_days_sample,weeks
0,2019-03-18 00:00:00,NaN,NaN,281.966725,1.862645e-09,0.225235,0.078516,-0.731408,4.957201e+06,37.167818,-4.190806,21,3
1,2019-03-18 01:00:00,NaN,NaN,281.286275,1.862645e-09,0.225362,0.175199,-0.802007,0.000000e+00,37.167818,-4.190806,20,3
2,2019-03-18 02:00:00,NaN,NaN,280.745240,1.862645e-09,0.225491,0.228708,-0.879847,0.000000e+00,37.167818,-4.190806,20,3
3,2019-03-18 03:00:00,NaN,NaN,280.297184,1.862645e-09,0.225625,-0.065142,-0.860041,0.000000e+00,37.167818,-4.190806,20,3
4,2019-03-18 04:00:00,NaN,NaN,279.898409,1.862645e-09,0.225754,-0.584523,-0.603643,0.000000e+00,37.167818,-4.190806,20,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,2019-04-07 20:00:00,NaN,NaN,283.115515,5.500701e-03,0.396745,0.535968,1.157173,9.465320e+06,37.167818,-4.190806,0,1
501,2019-04-07 21:00:00,NaN,NaN,283.039120,6.552307e-03,0.405312,0.989625,0.768043,9.465320e+06,37.167818,-4.190806,0,1
502,2019-04-07 22:00:00,NaN,NaN,282.950793,7.053099e-03,0.404607,1.199463,0.583405,9.465320e+06,37.167818,-4.190806,0,1
503,2019-04-07 23:00:00,NaN,NaN,282.849159,7.186549e-03,0.399597,1.125977,0.480936,9.465320e+06,37.167818,-4.190806,0,1


In [19]:
def get_meteo_variables(codparcela, date_sample, n_weeks, df_climatico, meteo_columns):

    fecha_end_slice = fecha_muestra - timedelta(7*n_weeks)

    df_climatico = df_climatico[(df_climatico["date"] >= fecha_end_slice) & (df_climatico["date"] <= fecha_muestra)]

    df_climatico["diff_days_sample"] = (fecha_muestra - df_climatico["date"]).dt.days

    df_climatico["weeks"] = df_climatico["diff_days_sample"].apply(group_days_in_weeks)


    resultados = df_datos_muestra.groupby("weeks")[VARIABLES].agg(operations)

    # # Renombrar las columnas según el patrón "variable_stat"
    # resultados.columns = [f'duracion_dias_{stat}' for stat in resultados.columns]

    resultados.columns = meteo_columns

    n_rows, n_column = resultados.shape

    flat_values = resultados.values.reshape(n_rows * n_column)

    resultados_columns = [f"{column}_{i}" for i in range(n_rows) for column in resultados.columns]

    df = pd.DataFrame([flat_values], columns=resultados_columns)


    resultados["codparcela"] = id_codparcela
    resultados["fecha"] = fecha_muestra

    return df
    
    

In [56]:
df

,codparcela,fecha,stl1_min_0,tp_min_0,swvl1_min_0,u10_min_0,v10_min_0,ssr_min_0,value_ndvi_min_0,value_fapar_min_0,...,value_ndvi_mean_3,value_fapar_mean_3,stl1_std_3,tp_std_3,swvl1_std_3,u10_std_3,v10_std_3,ssr_std_3,value_ndvi_std_3,value_fapar_std_3
0,024-00189-08-00,2021-03-15,278.057615,290.439009,283.933507,4.385224,1.862645e-09,0.000002,4.370231e-07,6.288143e-07,...,3.380596e+06,4.260860e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
